## Collect a Company's Patent Porfolio

This notebook will attempt to amass, for any given company, their entire worldwide patent porfolio.\*

### U.S. Portfolio

Collecting their U.S. portfolio will proceed in a few steps:

1. Collect all applications ever assigned to the company

    a. Those applications naming the company as an applicant
    b. Those assigned to the company via an assignment
    
2. Prune all applications assigned out of the company, as recorded in USPTO assignment records

\*NOTE: I say "attempt" because the notebook relies on the USPTO system, and the International Patent Document (INPADOC) service. Not all foreign jurisdictions participate in INPADOC, and not all INPADOC member states are as good at contributing to the system as we would like. So coverage is good, but not perfect.

First, we do our standard imports, and grab the USApplication and Assignment objects from patent_client

In [1]:
import pandas as pd
from patent_client import USApplication, Assignment

company_name = 'Avid Technology'

### Step 0: Confirm that the search retreives the correct company's applications

In [2]:
Assignment.objects.filter(assignee=company_name).values_list("pat_assignee_name", flat=True)[:30]
    

['AVID TECHNOLOGY LIMITED',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 ['AVID TECHNOLOGY INC.', 'AVID SYSTEMS, INC.'],
 ['AVID TECHNOLOGY INC.', 'AVID SYSTEMS, INC.'],
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.',
 'AVID TECHNOLOGY, INC.']

### Step 1.a: Collect all applications naming the company as applicant

In [3]:
applicant_apps = USApplication.objects.filter(first_named_applicant=company_name).values_list('appl_id', flat=True)
len(applicant_apps)

NotAvailableException: There is a USPTO problem

### Step 1.b: Collect all applications assigned to the company

In [ ]:
from itertools import chain
# The assigned apps is either a single value, or a list of values if more than one property was assigned

assigned_apps = [a.appl_id for a in chain.from_iterable(Assignment.objects.filter(assignee=company_name).values_list("properties", flat=True))]
len(set(assigned_apps))

And now we combine the two

In [ ]:
all_apps = list(applicant_apps) + assigned_apps
all_apps = list(set(all_apps)) # use set to deduplicate the list

print(f'Total U.S. Applications Ever Owned by {company_name}: {len(all_apps)}')

### Step 2: Collect all applications ever assigned out of the company

In [12]:
assignments_out = Assignment.objects.filter(assignor=company_name)
assigned_out_apps = list()

for assignment in assignments_out:
    if 'assignors interest' in assignment.conveyance_text.lower():
        a = assignment.appl_num
        if isinstance(a, list):
            assigned_out_apps += a
        else:
            assigned_out_apps.append(a)
            
len(assigned_out_apps)

48

### Step 3: Subtract one set from the other

In [14]:
owned_apps = list(set(all_apps) - set(assigned_out_apps))
len(owned_apps)


314

In [31]:
n_apps = owned_apps

### Step 4: Generate status report

In [ ]:
import pandas as pd
from patent_client.parser import parse

apps = USApplication.objects.filter(appl_id=owned_apps)

df = pd.DataFrame.from_records(apps.values(
    'appl_id', 
    'app_filing_date', 
    'app_early_pub_number',
    'app_early_pub_date',
    'patent_number',
    'patent_issue_date',
    'patent_title',
    'inventors__0__name',
    'applicants__0__name_line_one',
    'app_status',
    'app_status_date',
    'expiration__date',
    'expiration__parent_appl_id',
    'expiration__parent_app_filing_date',
    'expiration__pta_or_pte',
    'expiration__terminal_disclaimer_filed'
))

def display_num(number):
    if number:
        return parse(number).display()
    else:
        return number

#df['patent_title'] = df['patent_title'].apply(str.capitalize)
#df['appl_id'] = df['appl_id'].apply(display_num)
#df['app_early_pub_number'] = df['app_early_pub_number'].apply(display_num)
#df['patent_number'] = df['patent_number'].apply(display_num)
us_df = df
us_df.head()

> /Users/parkerhancock/Projects/patent_client/src/patent_client/uspto_peds.py(131)is_online()
-> raise NotAvailableException("There is a USPTO problem")


In [33]:
from collections import defaultdict
from itertools import chain

assignments = chain(Assignment.objects.filter(assignee=company_name), Assignment.objects.filter(assignor=company_name))

# Create a reverse index of the Assignments out by application number
assignment_index = defaultdict(set)
for assignment in assignments:
    for p in assignment.properties:
        us_app = p.us_application
        for parent in us_app.parents:
            assignment_index[parent.parent_appl_id].add(assignment)
        for child in us_app.children:
            assignment_index[child.child_appl_id].add(assignment)
        assignment_index[p.appl_id].add(assignment)

# Sort each entry in reverse chronological order, and filter out security interests
for appl_id, assignments in assignment_index.items():
    assignments = filter(lambda x: "security" not in x.conveyance_text.lower(), assignments)
    assignments = sorted(assignments, key=lambda x: x.recorded_date, reverse=True)
    assignment_index[appl_id] = list(assignments)

In [34]:
def annotate_with_assignments(row):
    assignments = assignment_index[row['appl_id']]
    if len(assignments):
        assignment = assignments[0]
        assignment_data = pd.Series(dict(
            current_assignee=assignment.pat_assignee_name,
            reel_frame=assignment.id,
            conveyance_type=assignment.conveyance_text,
            assignment_date=assignment.recorded_date
        ))
        return row.append(assignment_data)
    return row

us_df = us_df.apply(annotate_with_assignments, axis=1)[list(us_df.columns) + ['current_assignee', 'reel_frame', 'assignment_date', 'conveyance_type']]
us_df


,appl_id,app_filing_date,app_early_pub_number,app_early_pub_date,patent_number,patent_issue_date,patent_title,inventors__0__name,applicants__0__name_line_one,app_status,app_status_date,expiration__date,expiration__parent_appl_id,expiration__parent_app_filing_date,expiration__pta_or_pte,expiration__terminal_disclaimer_filed,current_assignee,reel_frame,assignment_date,conveyance_type
0,15806826,2017-11-08,US20180128073A1,2018-05-10,None,None,POWDER METAL GRIPPING ELEMENTS FOR SETTABLE DO...,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Non Final Action Mailed,2019-07-12,None,15672790,2017-08-09,0,False,"MAGNUM OIL TOOLS INTERNATIONAL, LTD",47305-684,2018-10-25,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
1,16109233,2018-08-22,US20180363415A1,2018-12-20,None,None,DOWNHOLE ASSEMBLY FOR SELECTIVELY SEALING OFF ...,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Notice of Allowance Mailed -- Application Rece...,2019-07-11,None,11949629,2007-12-03,0,False,MAGNUM OIL TOOLS INTERNATIONAL LTD.,43890-386,2017-10-18,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
2,15189090,2016-06-22,US20170030161A1,2017-02-02,10352125,2019-07-16,DOWNHOLE PLUG HAVING DISSOLVABLE METALLIC AND ...,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Patented Case,2019-06-26,None,13893205,2013-05-13,194,True,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",48724-60,2019-03-28,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
3,16265808,2019-02-01,US20190169951A1,2019-06-06,None,None,EXTENDED REACH PLUG HAVING DEGRADABLE ELEMENTS,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Docketed New Case - Ready for Examination,2019-06-24,None,13373223,2011-11-08,0,False,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",48724-60,2019-03-28,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
4,15697572,2017-09-07,US20180010413A1,2018-01-11,None,None,PLUG OF EXTENDED REACH,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Publications -- Issue Fee Payment Verified,2019-06-18,None,13373223,2011-11-08,0,True,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",48724-60,2019-03-28,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
5,15654156,2017-07-19,US20170314363A1,2017-11-02,None,None,DOWNHOLE ASSEMBLY FOR SELECTIVELY SEALING OFF ...,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Non Final Action Mailed,2019-04-30,None,11949629,2007-12-03,0,False,MAGNUM OIL TOOLS INTERNATIONAL LTD.,43890-386,2017-10-18,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
6,16274906,2019-02-13,US20190178045A1,2019-06-13,None,None,ELECTRICALLY POWERED SETTING TOOL AND PERFORAT...,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Docketed New Case - Ready for Examination,2019-04-23,None,14922969,2015-10-26,0,False,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",48919-845,2019-04-18,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
7,14922969,2015-10-26,US20160115753A1,2016-04-28,None,None,ELECTRICALLY POWERED SETTING TOOL AND PERFORAT...,None,"Magnum Oil Tools International, Ltd.",Abandonment for Failure to Correct Drawings/Oa...,2019-04-18,None,14922969,2015-10-26,0,False,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",48919-845,2019-04-18,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
8,16211847,2018-12-06,US20190169953A1,2019-06-06,None,None,MOLDED DEGRADABLE DOWNHOLE TOOL ELEMENTS,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Docketed New Case - Ready for Examination,2019-03-08,None,16211847,2018-12-06,0,False,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",47744-342,2018-12-11,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...
9,62625099,2018-02-01,None,None,None,None,INSERTS FOR INJECTION MOLDED DEGRADABLE DOWNHO...,None,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",Provisional Application Expired,2019-02-03,None,62625099,2018-02-01,0,False,"MAGNUM OIL TOOLS INTERNATIONAL, LTD.",47744-342,2018-12-11,ASSIGNMENT OF ASSIGNORS INTEREST (SEE DOCUMENT...


### Foreign Patent Portfolio

Now we will fetch the company's wordwide portfolio, as it appears in the Inpadoc system maintained by the EPO

In [35]:
from patent_client import Inpadoc
from patent_client.util import values_iterator

foreign = (p.document for p in Inpadoc.objects.filter(applicant=company_name) if p.country != "US")
values = ('country', 'application', 'publication', 'publication_date', 'title', 'applicants__0', 'inventors__0', 'legal__-1__description', 'legal__-1__date')
foreign_df = pd.DataFrame.from_records(values_iterator(foreign, *values))
foreign_df['title'] = foreign_df['title'].apply(lambda x: x.capitalize() if x else None)
foreign_df['status'] = foreign_df['legal__-1__description']
foreign_df['status_date'] = foreign_df['legal__-1__date']
foreign_df = foreign_df.drop(labels=['legal__-1__description', 'legal__-1__date'], axis=1)
foreign_df

,country,application,publication,publication_date,title,applicants__0,inventors__0,status,status_date
0,WO,US2019015461,WO2019148128A1,2019-08-01,Gas capable frangible disc barrier valve,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W,",None,None
1,CA,CA3026847,CA3026847A1,2019-06-06,Molded degradable downhole tool elements,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W. LYNN",None,None
2,CA,CA3021453,CA3021453A1,2019-04-18,"Split ring slips, slotted unibody slips, multi...","MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W. LYNN",None,None
3,CA,CA3035430,CA3035430A1,2014-06-26,Downhole tools having non-toxic degradable ele...,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, DERRICK,",EXAMINATION REQUEST,2019-03-14
4,CA,CA2985278,CA2985278A1,2018-05-08,Powder metal gripping elements for settable do...,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W. LYNN",None,None
5,CA,CA2975842,CA2975842A1,2018-02-09,Split ring sealing assemblies,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W. LYNN",None,None
6,CA,CA2937076,CA2937076A1,2017-01-24,Interventionless frangible disk isolation tool,"MAGNUM OIL TOOLS INTERNATIONAL, LLC","YONG, LAKHENA,",EXAMINATION REQUEST,2018-11-28
7,CA,CA2935508,CA2935508A1,2015-10-02,Downhole plug having dissolvable metallic and ...,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W. LYNN",EXAMINATION REQUEST,2019-01-28
8,CA,CA2910136,CA2910136A1,2016-04-24,Electrically powered setting tool and perforat...,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W., LYNN,",None,None
9,WO,US2015037636,WO2016003759A1,2016-01-07,Dissolvable aluminum downhole plug,"MAGNUM OIL TOOLS INTERNATIONAL, LTD","FRAZIER, W., LYNN",EP: PCT APPLICATION NON-ENTRY IN EUROPEAN PHASE,2017-07-26


And now, a status report:

And now we will take our results, and write to an excel file

In [36]:
writer = pd.ExcelWriter(f'{company_name}_portfolio.xlsx')
us_df.to_excel(writer, sheet_name='US Portfolio')
foreign_df.to_excel(writer, sheet_name='Foreign Portfolio')
writer.save()